In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_text
from sklearn.tree import plot_tree
from sklearn.datasets import load_iris
from sklearn.preprocessing import label_binarize
import pickle
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Car Sales.xlsx - car_data.csv'
df = pd.read_csv(file_path)
df.head()

,Car_id,Date,Customer Name,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price ($),Dealer_No,Body Style,Phone,Dealer_Region
0,C_CND_000001,1/2/2022,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,26000,06457-3834,SUV,8264678,Middletown
1,C_CND_000002,1/2/2022,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,19000,60504-7114,SUV,6848189,Aurora
2,C_CND_000003,1/2/2022,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500,38701-8047,Passenger,7298798,Greenville
3,C_CND_000004,1/2/2022,Giselle,Male,13500,Chrysler of Tri-Cities,Toyota,Celica,Overhead Camshaft,Manual,Pale White,14000,99301-3882,SUV,6257557,Pasco
4,C_CND_000005,1/2/2022,Grace,Male,1465000,Chrysler Plymouth,Acura,TL,DoubleÂ Overhead Camshaft,Auto,Red,24500,53546-9427,Hatchback,7081483,Janesville


In [ ]:
df.isnull().sum()

Car_id           0
Date             0
Customer Name    1
Gender           0
Annual Income    0
Dealer_Name      0
Company          0
Model            0
Engine           0
Transmission     0
Color            0
Price ($)        0
Dealer_No        0
Body Style       0
Phone            0
Dealer_Region    0
dtype: int64

In [ ]:
df['Customer Name'].value_counts()

Customer Name
Thomas           92
Emma             90
Lucas            88
Nathan           80
Louis            76
                 ..
Adelin            1
Zakarya           1
Paule             1
Noeline           1
Djamel Epoine     1
Name: count, Length: 3021, dtype: int64

In [ ]:
df["Customer Name"]=df["Customer Name"].fillna("Thomas")

In [ ]:
df.isnull().sum()

Car_id           0
Date             0
Customer Name    0
Gender           0
Annual Income    0
Dealer_Name      0
Company          0
Model            0
Engine           0
Transmission     0
Color            0
Price ($)        0
Dealer_No        0
Body Style       0
Phone            0
Dealer_Region    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Car_id         23906 non-null  object
 1   Date           23906 non-null  object
 2   Customer Name  23906 non-null  object
 3   Gender         23906 non-null  object
 4   Annual Income  23906 non-null  int64 
 5   Dealer_Name    23906 non-null  object
 6   Company        23906 non-null  object
 7   Model          23906 non-null  object
 8   Engine         23906 non-null  object
 9   Transmission   23906 non-null  object
 10  Color          23906 non-null  object
 11  Price ($)      23906 non-null  int64 
 12  Dealer_No      23906 non-null  object
 13  Body Style     23906 non-null  object
 14  Phone          23906 non-null  int64 
 15  Dealer_Region  23906 non-null  object
dtypes: int64(3), object(13)
memory usage: 2.9+ MB


In [ ]:
space_count = sum(s.count(' ') for s in df)
print("Total spaces:", space_count)

Total spaces: 5


In [ ]:
df.columns

Index(['Car_id', 'Date', 'Customer Name', 'Gender', 'Annual Income',
       'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color',
       'Price ($)', 'Dealer_No ', 'Body Style', 'Phone', 'Dealer_Region'],
      dtype='object')

In [ ]:
space_count = sum(s.count(' ') for s in df["Customer Name"])
print("Total spaces:", space_count)

Total spaces: 41


In [ ]:
df["Customer Name"].value_counts()

Customer Name
Thomas           93
Emma             90
Lucas            88
Nathan           80
Louis            76
                 ..
Adelin            1
Zakarya           1
Paule             1
Noeline           1
Djamel Epoine     1
Name: count, Length: 3021, dtype: int64

In [ ]:
df.columns

Index(['Car_id', 'Date', 'Customer Name', 'Gender', 'Annual Income',
       'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color',
       'Price ($)', 'Dealer_No ', 'Body Style', 'Phone', 'Dealer_Region'],
      dtype='object')

In [ ]:
fig = px.histogram(df,x="Company" , y="Price ($)" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of Price ($) by Company",
    xaxis_title="Company",
    yaxis_title="Price"
)
fig.show()

In [ ]:
df_genre_sales = df.groupby('Engine')['Price ($)'].sum().reset_index()
fig = px.pie(df_genre_sales, values='Price ($)', names='Engine', height=400)
fig.update_layout(
    title_text="Distribution of Engine by Price",
    showlegend=True
)

fig.show()


In [ ]:
df_genre_sales = df.groupby('Color')['Price ($)'].sum().reset_index()
fig = px.pie(df_genre_sales, values='Price ($)', names='Color', height=400)
fig.update_layout(
    title_text="Distribution of Color by Price",
    showlegend=True
)

fig.show()


In [ ]:
df_genre_sales = df.groupby('Gender')['Price ($)'].sum().reset_index()
fig = px.pie(df_genre_sales, values='Price ($)', names='Gender', height=400)
fig.update_layout(
    title_text="Distribution of Gender by Price",
    showlegend=True
)

fig.show()


In [ ]:
fig = px.histogram(df,x="Phone" , y="Price ($)" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of Price ($) by Phone",
    xaxis_title="Phone",
    yaxis_title="Price"
)
fig.show()

In [ ]:
df = df.drop(columns='Phone')

In [ ]:
df = df.drop(columns='Car_id')

In [ ]:
df = df.drop(columns='Date')

In [ ]:
df = df.drop(columns='Customer Name')

In [ ]:
df = df.drop(columns='Annual Income')

In [ ]:
df.columns

Index(['Gender', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission',
       'Color', 'Price ($)', 'Dealer_No ', 'Body Style', 'Dealer_Region'],
      dtype='object')

In [ ]:
df_genre_sales = df.groupby('Transmission')['Price ($)'].sum().reset_index()
fig = px.pie(df_genre_sales, values='Price ($)', names='Transmission', height=400)
fig.update_layout(
    title_text="Distribution of Transmission by Price",
    showlegend=True
)

fig.show()


In [ ]:
df_genre_sales = df.groupby('Dealer_Name')['Price ($)'].sum().reset_index()
fig = px.pie(df_genre_sales, values='Price ($)', names='Dealer_Name', height=400)
fig.update_layout(
    title_text="Distribution of Dealer_Name by Price",
    showlegend=True
)

fig.show()


In [ ]:
df_genre_sales = df.groupby('Dealer_No ')['Price ($)'].sum().reset_index()
fig = px.pie(df_genre_sales, values='Price ($)', names='Dealer_No ', height=400)
fig.update_layout(
    title_text="Distribution of Dealer_No  by Price",
    showlegend=True
)

fig.show()


In [ ]:
df_genre_sales = df.groupby('Body Style')['Price ($)'].sum().reset_index()
fig = px.pie(df_genre_sales, values='Price ($)', names='Body Style', height=400)
fig.update_layout(
    title_text="Distribution of Body Style by Price",
    showlegend=True
)

fig.show()


In [ ]:
df_genre_sales = df.groupby('Dealer_Region')['Price ($)'].sum().reset_index()
fig = px.pie(df_genre_sales, values='Price ($)', names='Dealer_Region', height=400)
fig.update_layout(
    title_text="Distribution of Dealer_Region by Price",
    showlegend=True
)

fig.show()


In [ ]:
df = df.drop(columns='Dealer_No ')

In [ ]:
df.columns

Index(['Gender', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission',
       'Color', 'Price ($)', 'Body Style', 'Dealer_Region'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Gender         23906 non-null  object
 1   Dealer_Name    23906 non-null  object
 2   Company        23906 non-null  object
 3   Model          23906 non-null  object
 4   Engine         23906 non-null  object
 5   Transmission   23906 non-null  object
 6   Color          23906 non-null  object
 7   Price ($)      23906 non-null  int64 
 8   Body Style     23906 non-null  object
 9   Dealer_Region  23906 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.8+ MB


In [ ]:
features_to_encode = ['Gender', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Body Style', 'Dealer_Region']

# Initialize the LabelEncoder
encoder = LabelEncoder()

# Apply label encoding to each feature
for feature in features_to_encode:
    df[feature] = encoder.fit_transform(df[feature])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Gender         23906 non-null  int64
 1   Dealer_Name    23906 non-null  int64
 2   Company        23906 non-null  int64
 3   Model          23906 non-null  int64
 4   Engine         23906 non-null  int64
 5   Transmission   23906 non-null  int64
 6   Color          23906 non-null  int64
 7   Price ($)      23906 non-null  int64
 8   Body Style     23906 non-null  int64
 9   Dealer_Region  23906 non-null  int64
dtypes: int64(10)
memory usage: 1.8 MB


In [ ]:
feature_cols =['Gender', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission','Color','Body Style', 'Dealer_Region']
X = df[feature_cols]
y = df['Price ($)']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), feature_cols)
    ])

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('poly', poly),
    ('model', LinearRegression())
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['Gender', 'Dealer_Name',
                                                   'Company', 'Model', 'Engine',
                                                   'Transmission', 'Color',
                                                   'Body Style',
                                                   'Dealer_Region'])])),
                ('poly', PolynomialFeatures(include_bias=False)),
                ('model', LinearRegression())])

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
mean_y = np.mean(y_test)
total_variance = np.sum((y_test - mean_y) ** 2)

# Calculate residual sum of squares (RSS)
residual_sum_of_squares = np.sum((y_test - y_pred) ** 2)

# Calculate explained variance as a percentage
explained_variance_percentage = (1 - residual_sum_of_squares / total_variance) * 100

print(f'Explained Variance Percentage: {explained_variance_percentage:.2f}%')

Explained Variance Percentage: 60.12%


In [ ]:
# dt = DecisionTreeClassifier(random_state=42)
# dt.fit(X_train, y_train)

In [ ]:
# y_pred = dt.predict(X_test)

In [ ]:
# accuracyDecesionTree = accuracy_score(y_test, y_pred)
# accuracyDecesionTree=accuracyDecesionTree*100

In [ ]:
# print("Accuracy:", accuracyDecesionTree)

In [ ]:
lg = LogisticRegression()
lg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [ ]:
y_predict= lg.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_predict)
accuracy_log_reg=accuracy*100
print(accuracy_log_reg)

5.729820158929319


In [ ]:
y_prob = lg.predict_proba(X_test)[:, 1]

In [ ]:
# plt.figure(figsize=(20,10))
# plot_tree(dt, filled=True, feature_names=X.columns)
# plt.show()

In [ ]:
# rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_clf.fit(X_train, y_train)

In [ ]:
# rf_initial_accuracy = rf_clf.score(X_test, y_test)
# rf_initial_accuracy=rf_initial_accuracy*100
# print(rf_initial_accuracy)

In [ ]:
model = SVC()
model.fit(X_train, y_train)

SVC()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracySVM = accuracy_score(y_test, y_pred)*100
print('SVM Accuracy: ', accuracySVM)

SVM Accuracy:  5.855290673358427


In [ ]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracyKNN = accuracy_score(y_test, y_pred)*100
print('KNN Accuracy: ', accuracyKNN)

KNN Accuracy:  23.02383939774153


In [ ]:
model = SVR()
model.fit(X_train, y_train)

SVR()

In [ ]:
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
accuracySVR = 100 - mape

print(f'Accuracy: {accuracySVR:.2f}%')

Accuracy: 67.64%


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
accuracyLN = 100 - mape
print(f'Accuracy: {accuracyLN:.2f}%')

Accuracy: 53.07%


In [ ]:
model = SVR()
model.fit(X_train, y_train)

SVR()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
accuracySVR = 100 - mape

print(f'Accuracy: {accuracySVR:.2f}%')

Accuracy: 64.40%


In [ ]:
gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)

GradientBoostingRegressor()

In [ ]:
y_pred = gb_model.predict(X_test)

In [ ]:
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
accuracyGradientBoostingRegressor = 100 - mape

print(f'Accuracy: {accuracyGradientBoostingRegressor:.2f}%')

Accuracy: 59.86%


In [ ]:
lgb_reg = lgb.LGBMRegressor()
lgb_reg.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003884 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 233
[LightGBM] [Info] Number of data points in the train set: 19124, number of used features: 9
[LightGBM] [Info] Start training from score 28158.840358


LGBMRegressor()

In [ ]:
rf_clf=lgb.LGBMRegressor()

In [ ]:
y_pred = lgb_reg.predict(X_test)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

accuracyLGBMRegressor = 100 - mean_absolute_percentage_error(y_test, y_pred)
print(f"Accuracy: {accuracyLGBMRegressor:.2f}%")

Accuracy: 72.86%


In [ ]:
def compare_algorithms_accuracy(df, feature_cols, test_size=0.2, random_state=42):
    allAcc = {'Logistic Regression Accuracy': accuracy_log_reg,
      'polynominal regression':explained_variance_percentage,
      'KNN accuracy':accuracyKNN,
      'SVM accuracy':accuracySVM,
      'accuracy SVR':accuracySVR,
      'LinearRegression aaccuracy': accuracyLN,
      'GradientBoostingRegressor accuracy':accuracyGradientBoostingRegressor,
      'LGBMRegressor accuracy':accuracyLGBMRegressor,}

    highest_accuracy = max(allAcc, key=allAcc.get)
    return f"The algorithm with the highest accuracy is: {highest_accuracy} with an accuracy of {allAcc[highest_accuracy]:.2f}%"
result = compare_algorithms_accuracy(df, feature_cols)
print(result)

The algorithm with the highest accuracy is: LGBMRegressor accuracy with an accuracy of 72.86%


In [ ]:
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf_clf, file)